In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def total_portfolio(weights, portfolio):
    portfolio = (pd.DataFrame(
    np.sum(
        np.multiply(
            portfolio[weights.index.to_list()].values,
            np.transpose(weights['Weights'].values)), 
        axis=1),
    index=portfolio.index,
    columns=['portfolio'])
    )
    return portfolio

In [ ]:
def beta_function(weights, portfolio, benchmark):
    """
    Ouput:
        Métrica Beta CAPM
    Inputs:
        portfolio (type dataframe pandas): Retornos de la cartera
        benchmark (type dataframe pandas): Retornos del índice de referencia
    """
    portfolio = total_portfolio(weights, portfolio)
    # Concatenar el portfolio y el índice
    join = pd.concat((portfolio, benchmark), axis=1).dropna()
    # Covarianza y varianza
    cov, var = np.cov(join, rowvar=False)[1]
    return cov / var

In [ ]:
def alpha_function(weights, portfolio, benchmark, timeframe=252):
    """
    Ouput:
        Métrica Alpha CAPM
    Inputs:
        portfolio (type dataframe pandas): Retornos de la cartera
        benchmark (type dataframe pandas): Retornos del índice de referencia
    """
    # Calcula la beta
    beta = beta_function(weights, portfolio, benchmark)
    portfolio = total_portfolio(weights, portfolio)
    mean_stock_return = portfolio.values.mean() * timeframe
    mean_market_return = benchmark.values.mean() * timeframe
    return mean_stock_return - beta * mean_market_return

In [ ]:
def sharpe_function(weights, portfolio, timeframe=252):
    """
    Output:
        Métrica del ratio de Sharpe
    Inputs:
        portfolio (type dataframe pandas): Retornos de la cartera
        timeframe (type int): Factor de anualización
    """
    portfolio = total_portfolio(weights, portfolio)
    # Media y desviación típica de los retornos anualizados 
    portfolio = portfolio.values
    mean = portfolio.mean() * timeframe
    std = portfolio.std() * np.sqrt(timeframe)
    return mean / std

In [ ]:
def sortino_function(weights, portfolio, timeframe=252):
    """
    Output:
        Métrica del ratio de Sharpe
    Inputs:
        portfolio (type dataframe pandas): Retornos de la cartera
        timeframe (type int): Factor de anualización
    """
    portfolio = total_portfolio(weights, portfolio)
    # Tomar valores a la baja
    portfolio = portfolio.values
    downward = portfolio[portfolio < 0]
    mean = portfolio.mean() * timeframe
    std = downward.std() * np.sqrt(timeframe)
    return mean / std

In [ ]:
def drawdown_function(weights, portfolio):
    """
    Output: 
        Drawdown
    Inputs:
        portfolio (type dataframe pandas): Retornos de la cartera
    """
    portfolio = total_portfolio(weights, portfolio)
    # Calcular el producto acumulado de los rendimientos
    cum_rets = (portfolio + 1).cumprod()
    # Calcular el máximo producto acumulado
    running_max = np.maximum.accumulate(cum_rets.dropna())
    running_max[running_max < 1] = 1
    # Calcular el drawdown
    drawdown = (cum_rets / running_max - 1)
    return drawdown

In [ ]:
def VaR_function(theta, mu, sigma):
    """
    Output:
        VaR
    Inputs:
        theta (type float): umbral del error en %
        mu (type float): retorno esperado de la cartera
        sigma (type float): volatilidad de la cartera
    """
    # Número de simulaciones
    n = 100000
    # Encontrar los valores para el umbral de error theta
    t = int(n * theta)
    # Crear un vector con n simulaciones de la ley normal
    vec = pd.DataFrame(np.random.normal(mu, sigma, size=(n,)),
                       columns=['Simulations'])
    # Ordena los valores y encuentra el valor theta%
    var = vec.sort_values(by="Simulations").iloc[t].values[0]
    return var

In [ ]:
def cVaR_function(theta, mu, sigma):
    """
    Output:
        cVaR
    Inputs:
        theta (type float): umbral del error en %
        mu (type float): retorno esperado de la cartera
        sigma (type float): volatilidad de la cartera
    """
    # Número de simulaciones
    n = 100000
    # Encontrar los valores para el umbral de error theta
    t = int(n * theta)
    # Crear un vector con n simulaciones de la ley normal
    vec = pd.DataFrame(np.random.normal(mu, sigma, size=(n,)),
                       columns=['Simulations'])
    # Ordena los valores y encuentra el valor theta%
    cvar = vec.sort_values(by="Simulations").iloc[0:t, :].mean().values[0]
    return cvar

In [ ]:
def CR_function(weights, portfolio, benchmark):
    """
    Output:
        Métrica de la contribución al riesgo
    Input:
        weights (type pandas.DataFrame): Pesos de la cartera
        portfolio (type pandas.DataFrame): Retornos de los activos
        benchmark (type pandas.DataFrame): Retornos del índice de referencia
    """
    # Calcular la contribución al riesgo de cada activo
    crs = []
    for asset in weights.index.to_list():
        cr = beta_function(weights.loc[asset].to_frame().T, portfolio[asset].to_frame(), benchmark) * weights.loc[asset]
        crs.append(cr)
    # Normalizar por la suma de la contribución al riesgo
    return crs / np.sum(crs)

In [ ]:
def backtest_function(weights, portfolio, benchmark, timeframe=252):
    """
    Output:
        Métricas
    Input:
        weights (type pandas.DataFrame): Pesos de la cartera
        portfolio (type pandas.DataFrame): Retornos de los activos
        benchmark (type pandas.DataFrame): Retornos del índice de referencia
        timeframe (type int): Factor de anualización
    """
    # Calcular la BETA
    beta = beta_function(weights, portfolio, benchmark)
    # Calcular el ALPHA
    alpha = alpha_function(weights, portfolio, benchmark)
    # Calcular el Ratio de Sharpe
    sharpe = sharpe_function(weights, portfolio)
    # Calcular el Ratio de Sortino
    sortino = sortino_function(weights, portfolio)
    # Calcular el Drawdown
    drawdown = drawdown_function(weights, portfolio)
    # Calcular el CR
    cr = CR_function(weights, portfolio, benchmark)
    # Calcular el VaR
    portfolio = total_portfolio(weights, portfolio)
    mean = portfolio.mean() * timeframe
    std = portfolio.std() * np.sqrt(timeframe)
    theta = 0.05
    var = VaR_function(theta, mean, std) * 100.
    # Calcular el cVaR
    cvar = cVaR_function(theta, mean, std) * 100.

    # Print de los resultados
    portfolio_weights = "\n\t" + "\n\t".join(f'{index}: {item*100:.2f}%' for index, item in weights['Weights'].items())
    print(f"""
    __________________________________________________
    Portfolio: {portfolio_weights}
    __________________________________________________
    Beta: {np.round(beta, 3)}
    Alpha: {np.round(alpha, 3)}
    Sharpe: {np.round(sharpe, 3)}
    Sortino: {np.round(sortino, 3)}
    ___________________________________________________
    VaR: {np.round(var, 3)}
    cVaR: {np.round(cvar, 3)}
    ___________________________________________________
    """)
    columns = weights.index.to_list()
    # Visualizaciones gráficas
    plt.figure(figsize=(10, 6))
    plt.plot(portfolio.cumsum().sum(axis=1)*100.)
    plt.title("Retornos acumulados", size=15)
    plt.ylabel("%", size=15)
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.fill_between(drawdown.index, drawdown.iloc[:, 0]*100., 0, color='#E95751')
    plt.title("Drawdown", size=15)
    plt.ylabel("%", size=15)
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.bar(columns, cr.flatten()*100., color='#B96553')
    plt.title('Contribución al riesgo del portfolio', size=15)
    plt.ylabel('%', size=15)
    plt.show()
